In [43]:
def nbVar(ne,nj):
    return (nj-1)*ne*ne+(ne-1)*ne+(ne-2)+1

In [44]:
nbVar(4,4)

63

In [181]:
def codage(ne,nj,j,x,y):
    return (j*(ne**2))+(x*ne)+y+1

In [182]:
def decodage(ne,nj,k):
    y = (k-1) % ne
    x = (k-y-1)%(ne**2)/ne
    j = (k-1-x*ne-y)/(ne**2)
    return j,x,y

In [183]:
codage(4,4,3,2,1)

58

In [184]:
decodage(4,4,58)

(3.0, 2.0, 1)

In [185]:
def clauseAuMoins(l):
    return ' '.join(map(str,l)) + ' 0'

In [186]:
clauseAuMoins([1,2,3])

'1 2 3 0'

In [187]:
def clauseAuPlus(l):
    tmp = ''
    for i in range(len(l)):
        for j in range(i+1, len(l)):
            tmp += str(-l[i])+' '+str(-l[j])+' 0\n'
    return tmp[:-1]

In [188]:
print(clauseAuPlus([1,2,3]))

-1 -2 0
-1 -3 0
-2 -3 0


In [189]:
def encoderC1(ne, nj):
    res = ''
    for j in range(nj):
        for e1 in range(ne):
            l = []
            for e2 in range(ne):
                if e1!=e2:
                    v1 = codage(ne,nj,j,e1,e2)
                    v2 = codage(ne,nj,j,e2,e1)
                    l.append(v1)
                    l.append(v2)
            if l != []:
                res += clauseAuPlus(l)+'\n'
    return res[:-1]

In [190]:
len(encoderC1(3,4).split('\n'))

72

In [197]:
def encoderC2(ne, nj):
    res = ''
    for e1 in range(ne):
        for e2 in range(ne):
            if (e1 !=e2 ):
                aller = []
                retour = []
                for j in range(nj):
                    allerVal = codage(ne,nj,j,e1,e2)
                    retourVal = codage(ne,nj,j,e2,e1)
                    aller.append(allerVal)
                    retour.append(retourVal)
                if aller != [] and retour != []:
                    res += clauseAuPlus(aller)+'\n'
                    res += clauseAuPlus(retour)+'\n'
                    res += clauseAuMoins(aller)+'\n'
                    res += clauseAuMoins(retour)+'\n'
    return res[:-1]

In [198]:
len(encoderC2(3,4).split('\n'))

84

In [199]:
def encoder(ne, nj):
    return encoderC1(ne, nj) +'\n'+ encoderC2(ne, nj)

In [200]:
len(encoder(3,4).split('\n'))

156

In [201]:
def createCNF(nom, ne, nj):
    contraintes = encoder(ne, nj)
    with open(nom, "w") as f:
        f.write("p cnf " + str(nbVar(ne, nj)) + " " + str(len(contraintes.split('\n'))) + "\n"+contraintes)
    print("fichier "+nom+" créé.")

In [203]:
createCNF("e3q2.cnf", 3, 6)

fichier e3q2.cnf créé.


In [230]:
def decoderCNF(fichier, ne, nj):
    with open(fichier, "r") as f:
        line = f.read()
    line = line.split(' ')[:-1]
    line = [decodage(ne, nj, int(i)) for i in line if int(i) > 0]
    return line

In [231]:
decoderCNF("e3q2_output.cnf", 3, 6)

[(0.0, 1.0, 0),
 (1.0, 0.0, 2),
 (2.0, 2.0, 0),
 (3.0, 2.0, 1),
 (4.0, 0.0, 1),
 (5.0, 1.0, 2)]

In [235]:
import os
def planning(nom, ne, nj):
    equipes = {0:"PSG", 1:"OM", 2:"OL", 3:"MONACO", 4:"BORDEAUX", 5:"STRASBOURG"}
    createCNF("tmp.cnf", ne, nj)
    cmd = "./glucose_static tmp.cnf "+nom
    os.system(cmd)
    line = decoderCNF(nom, ne, nj)
    for (j,e1,e2) in line:
        print("jour:",int(j),"equipe n°",equipes[int(e1)],"vs","equipe n°",equipes[int(e2)])

In [237]:
planning("e3q2_output.cnf", 4, 6)

jour: 0 equipe n° PSG vs equipe n° MONACO
jour: 0 equipe n° OL vs equipe n° MONACO
jour: 1 equipe n° OL vs equipe n° PSG
jour: 2 equipe n° PSG vs equipe n° OL
jour: 2 equipe n° OM vs equipe n° OM
jour: 3 equipe n° PSG vs equipe n° OL


0